In [21]:
# basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint

# plot
import seaborn as sns
import matplotlib.pyplot as plt

# preprocess
#from sklearn.preprocessing import LabelEncoder

# model
from sklearn.model_selection import KFold
import lightgbm as lgb

# optimization
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error as MSE

# other
import pickle
import time
import datetime
#import sys
#import gc

In [116]:
train = pd.read_pickle('../features/feature_train_2020-11-07-08-04-37_treated.pkl')
test = pd.read_pickle('../features/feature_test_2020-11-07-08-04-37_treated.pkl')
sample_sub = pd.read_csv('../data/input/sample_submit.csv', header=None, names=['id', 'mpg'])

In [14]:
train

,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car_name_code,car_brand_code
0,0,23.059782,6,140,110.0,2815,17.977429,80,1,33,6
1,3,17.674521,8,350,150.0,4456,13.514535,72,1,39,6
2,4,17.136353,8,302,140.0,2774,13.209912,79,1,59,11
3,7,22.664666,6,400,85.0,2190,15.196381,71,1,72,16
4,9,17.872018,8,429,220.0,2245,9.621400,70,1,47,8
...,...,...,...,...,...,...,...,...,...,...,...
495,981,22.798447,4,140,148.0,2835,13.477573,82,1,25,5
496,983,35.173640,4,97,67.0,2234,17.542681,80,3,69,15
497,994,17.825448,8,302,220.0,2774,15.177189,76,1,81,20
498,995,28.545147,4,97,150.0,2130,13.324669,70,1,31,5


In [67]:
class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params.update(self.params)
        num_round = 10
        dtrain = lgb.Dataset(tr_x, tr_y)
        dvalid = lgb.Dataset(va_x, va_y)
        self.model = lgb.train(
            params, 
            dtrain, 
            num_boost_round=num_round,
            verbose_eval=True,
            valid_sets=dvalid
        )
        
    def predict(self, x):
        pred = self.model.predict(x, num_iteration=self.model.best_iteration)
        return pred

In [96]:
def score(params):
    params['max_depth'] = int(params['max_depth'])
    
    model = Model(params)
    model.fit(X_train, Y_train, X_valid, Y_valid)
    Y_pred = model.predict(X_valid)
    score = np.sqrt(MSE(y_true=Y_valid, y_pred=Y_pred))
    print(f'params: {params}, score:{score:.4f}')
    
    history.append((model, score))
    return {'loss':score, 'status':STATUS_OK}

In [78]:
params = {
    'objective':'regression',
    #'seed':71,
    'boosting_type':'gbdt',
    'metrics': {'l2'},
    #'min_child_weight':1,
    'max_depth':5,
    #'num_leaves':31,
    'num_boost_round':100, 
    'gamma':0.0,
    #'alpha':0.0,
    #'lambda':1.0,
    'learning_rate':0.1,
    #'feature_fraction':0.9, # colsample_bytree
    #'bagging_fraction':0.8, # subsample
    'bagging_freq':5
}

param_space = {
    'alpha' : hp.loguniform(
        label='alpha', 
        low=np.log(1e-8), 
        high=np.log(1.0)
    ),
    'bagging_fraction': hp.quniform(
        label='bagging_fraction', low=0.6, high=0.95, q=0.05
    ),
    'feature_fraction': hp.quniform(
        label='feature_fraction', low=0.6, high=0.95, q=0.05
    ),
    'gamma': hp.quniform(
        label='gamma', low=0.1, high=0.4, q=0.1
    ),
    'lambda' : hp.loguniform(
        label='lambda', 
        low=np.log(1e-6), 
        high=np.log(10.0)
    ),
    'max_depth': hp.quniform(
        label='max_depth', low=3, high=9, q=1
    ),
    'min_child_weight': hp.quniform(
        label='min_child_weight', low=1, high=5, q=1
    )
}

In [106]:
history = []
best = []

train_X = train.drop('mpg', axis=1)
train_y = train['mpg']

for i in range(5):
    kf = KFold(n_splits=4, shuffle=True, random_state=i)
    for tr_idx, va_idx in kf.split(train_X):
        X_train = train.drop('mpg', axis=1).loc[tr_idx]
        Y_train = train.loc[tr_idx, 'mpg']
        X_valid = train.drop('mpg', axis=1).loc[va_idx]
        Y_valid = train.loc[va_idx, 'mpg']

        fmin(
            fn=score,
            space=param_space, 
            algo=tpe.suggest, 
            trials=Trials(),
            max_evals=10
        )

        history = sorted(history, key=lambda tpl:tpl[1])
        best.append(history[0])

[1]	valid_0's l2: 50.4036                                                                                           
[2]	valid_0's l2: 44.6299                                                                                           
[3]	valid_0's l2: 40.2712                                                                                           
[4]	valid_0's l2: 36.4479                                                                                           
[5]	valid_0's l2: 33.2165                                                                                           
[6]	valid_0's l2: 30.0307                                                                                           
[7]	valid_0's l2: 27.5401                                                                                           
[8]	valid_0's l2: 25.3507                                                                                           
[9]	valid_0's l2: 23.2048                                       

D:\app-data\miniconda-dir\envs\ml\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[84]	valid_0's l2: 11.5884                                                                                          
[85]	valid_0's l2: 11.5917                                                                                          
[86]	valid_0's l2: 11.5627                                                                                          
[87]	valid_0's l2: 11.5265                                                                                          
[88]	valid_0's l2: 11.5057                                                                                          
[89]	valid_0's l2: 11.546                                                                                           
[90]	valid_0's l2: 11.5817                                                                                          
[91]	valid_0's l2: 11.5419                                                                                          
[92]	valid_0's l2: 11.5366                                      

D:\app-data\miniconda-dir\envs\ml\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[96]	valid_0's l2: 15.4633                                                                                          
[97]	valid_0's l2: 15.3953                                                                                          
[98]	valid_0's l2: 15.392                                                                                           
[99]	valid_0's l2: 15.3592                                                                                          
[100]	valid_0's l2: 15.3531                                                                                         
params: {'alpha': 0.6165289401817773, 'bagging_fraction': 0.65, 'feature_fraction': 0.6000000000000001, 'gamma': 0.30000000000000004, 'lambda': 0.08737961760981346, 'max_depth': 7, 'min_child_weight': 3.0}, score:3.9183
[1]	valid_0's l2: 52.8584                                                                                           
[2]	valid_0's l2: 49.2732                                                     

...

[84]	valid_0's l2: 10.6991                                                                                          
[85]	valid_0's l2: 10.6958                                                                                          
[86]	valid_0's l2: 10.6151                                                                                          
[87]	valid_0's l2: 10.5972                                                                                          
[88]	valid_0's l2: 10.5343                                                                                          
[89]	valid_0's l2: 10.4918                                                                                          
[90]	valid_0's l2: 10.4684                                                                                          
[91]	valid_0's l2: 10.4124                                                                                          
[92]	valid_0's l2: 10.4131                                      

D:\app-data\miniconda-dir\envs\ml\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[76]	valid_0's l2: 9.95811                                                                                          
[77]	valid_0's l2: 9.94291                                                                                          
[78]	valid_0's l2: 9.87181                                                                                          
[79]	valid_0's l2: 9.86194                                                                                          
[80]	valid_0's l2: 9.8636                                                                                           
[81]	valid_0's l2: 9.77407                                                                                          
[82]	valid_0's l2: 9.69647                                                                                          
[83]	valid_0's l2: 9.69924                                                                                          
[84]	valid_0's l2: 9.67459                                      

D:\app-data\miniconda-dir\envs\ml\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[96]	valid_0's l2: 10.7133                                                                                          
[97]	valid_0's l2: 10.7542                                                                                          
[98]	valid_0's l2: 10.768                                                                                           
[99]	valid_0's l2: 10.8208                                                                                          
[100]	valid_0's l2: 10.8415                                                                                         
params: {'alpha': 0.7967438337919024, 'bagging_fraction': 0.75, 'feature_fraction': 0.9500000000000001, 'gamma': 0.2, 'lambda': 0.00011042816050195725, 'max_depth': 3, 'min_child_weight': 2.0}, score:3.2926
100%|███████████████████████████████████████████████| 10/10 [00:02<00:00,  4.36trial/s, best loss: 3.13507322701374]


In [107]:
for i in range(len(best)):
    pprint.pprint(best[i][0].params)

{'alpha': 2.527803102483125e-07,
 'bagging_fraction': 0.9,
 'feature_fraction': 0.9500000000000001,
 'gamma': 0.30000000000000004,
 'lambda': 0.19755571923119897,
 'max_depth': 9,
 'min_child_weight': 2.0}
{'alpha': 1.938840413735587e-06,
 'bagging_fraction': 0.6000000000000001,
 'feature_fraction': 0.9,
 'gamma': 0.30000000000000004,
 'lambda': 0.00010750267618840512,
 'max_depth': 5,
 'min_child_weight': 3.0}
{'alpha': 1.938840413735587e-06,
 'bagging_fraction': 0.6000000000000001,
 'feature_fraction': 0.9,
 'gamma': 0.30000000000000004,
 'lambda': 0.00010750267618840512,
 'max_depth': 5,
 'min_child_weight': 3.0}
{'alpha': 8.733859317606222e-06,
 'bagging_fraction': 0.9500000000000001,
 'feature_fraction': 0.65,
 'gamma': 0.30000000000000004,
 'lambda': 0.07176630610604924,
 'max_depth': 8,
 'min_child_weight': 4.0}
{'alpha': 8.733859317606222e-06,
 'bagging_fraction': 0.9500000000000001,
 'feature_fraction': 0.65,
 'gamma': 0.30000000000000004,
 'lambda': 0.07176630610604924,
 'max

In [131]:
dt = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
submission = pd.DataFrame({'id':sample_sub.id})

for i in range(len(best)):
    submission = pd.concat(
        [submission, pd.Series(
            best[i][0].predict(test.drop('mpg', axis=1)),
            name='model_'+str(i)+'_pred'
        )],
        axis=1
    )
    
    pickle.dump(
        best[i][0],
        open(
            '../models/model_' + dt + '_lightgbm_' + str(i).zfill(2) + '.pickle',
            'wb'
        )
    )

In [133]:
submission['mean'] = submission.iloc[:, 1:].mean(axis='columns')

# save predictions for an ensemble
pickle.dump(
    submission, 
    open('../logs/test_preds' + dt + '.pickle', 'wb')
)

# create submission file
submission[['id', 'mean']].to_csv('../data/output/sub_' + dt + '_lightgbm.csv', header=False, index=False)